**Paper Link: ** [Generating Diverse and Consistent QA pairs from Contexts with HCVAE](https://aclanthology.org/2020.acl-main.20.pdf)

# Install Packages

In [ ]:
!git clone https://github.com/seanie12/Info-HCVAE.git

Cloning into 'Info-HCVAE'...
remote: Enumerating objects: 227, done.
remote: Counting objects: 100% (99/99), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 227 (delta 58), reused 59 (delta 25), pack-reused 128
Receiving objects: 100% (227/227), 66.83 MiB | 31.99 MiB/s, done.
Resolving deltas: 100% (113/113), done.


In [ ]:
!pip install torch-scatter -f https://data.pyg.org/whl/torch-1.10.0+cu111.html

Looking in links: https://data.pyg.org/whl/torch-1.10.0+cu111.html
     |████████████████████████████████| 7.9 MB 8.2 MB/s 


In [ ]:
!pip install transformers json-lines --quiet

     |████████████████████████████████| 3.8 MB 8.3 MB/s 
     |████████████████████████████████| 67 kB 5.3 MB/s 
     |████████████████████████████████| 6.5 MB 34.0 MB/s 
     |████████████████████████████████| 596 kB 41.2 MB/s 
     |████████████████████████████████| 895 kB 25.7 MB/s 


# Get Data

In [ ]:
%cd Info-HCVAE

/content/Info-HCVAE


In [ ]:
!mkdir data
!mkdir data/squad
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json -O ./data/squad/train-v1.1.json
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json -O ./data/squad/my_dev.json
# !wget https://zenodo.org/record/4557874/files/train_bangla_samples_fixed_preprocessed.json -O ./data/squad/train-v1.1.json
# !wget https://zenodo.org/record/4557874/files/valid_bangla_samples_fixed_preprocessed.json -O ./data/squad/my_dev.json

--2022-03-09 05:14:02--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30288272 (29M) [application/json]
Saving to: ‘./data/squad/train-v1.1.json’

./data/squad/train- 100%[===================>]  28.88M  --.-KB/s    in 0.1s    

2022-03-09 05:14:03 (221 MB/s) - ‘./data/squad/train-v1.1.json’ saved [30288272/30288272]

--2022-03-09 05:14:03--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4854279 (4.6M) [application/json]
Saving to: ‘./data/squa

In [ ]:
import json
def remove_impossible_questions(prevfilename,filename):
    f= open(prevfilename)
    data = json.load(f)
    for i in range(len(data['data'])):
        for j in range(len(data['data'][i]['paragraphs'])): # shobgula context pailam
            data['data'][i]['paragraphs'][j]['qas'] = [x for x in data['data'][i]['paragraphs'][j]['qas'] if len(x['answers'])!=0]
        
    out_file = open(filename, "w")
    json.dump(data, out_file, indent = 6)
    out_file.close()
    f.close()

In [ ]:
train_new = remove_impossible_questions('/content/Info-HCVAE/data/squad/train-v1.1.json',"/content/Info-HCVAE/data/squad/train-v1.1-new.json")

In [ ]:
my_dev_new = remove_impossible_questions('/content/Info-HCVAE/data/squad/my_dev.json',"/content/Info-HCVAE/data/squad/my_dev-new.json")

In [ ]:
!rm -r '/content/Info-HCVAE/data/squad/my_dev.json'
!rm -r '/content/Info-HCVAE/data/squad/train-v1.1.json'
!mv '/content/Info-HCVAE/data/squad/my_dev-new.json' '/content/Info-HCVAE/data/squad/my_dev.json'
!mv '/content/Info-HCVAE/data/squad/train-v1.1-new.json' '/content/Info-HCVAE/data/squad/train-v1.1.json'

# Import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd vae

/content/Info-HCVAE/vae


In [ ]:
import argparse
import os
import random

import numpy as np
import torch
from tqdm.notebook import tqdm, trange
from transformers import BertTokenizer

# from eval import eval_vae
# from trainer import VAETrainer
from utils import batch_to_device, get_harv_data_loader
from torch.utils.data import DataLoader, TensorDataset
from squad_utils import (convert_examples_to_features_answer_id,
                         convert_examples_to_harv_features,
                         convert_examples_to_features,
                         read_squad_examples)
# from utils import get_squad_data_loader


In [ ]:
args = {}

args['seed'] = 1004
args['debug'] = False
args['train_dir'] = '../data/squad/train-v1.1.json'
args['dev_dir'] = '../data/squad/my_dev.json'
args['max_c_len'] = 384 # max context length
args['max_q_len'] = 64 # max query length
args['model_dir'] = "save/vae-checkpoint" # "../save/vae-checkpoint"
args['load_checkpoint'] = "/content/drive/MyDrive/Thesis/HCVAE/save_english/save/vae-checkpoint/best_f1_model.pt"
args['epochs'] = 10
args['lr'] = 1e-3
args['batch_size'] = 32
args['weight_decay'] = 0.0
args['clip'] = 5.0 # max grad norm
args['bert_model'] = "bert-base-uncased" # "Kowsher/bangla-bert" # "mrm8488/bert-small-finetuned-squadv2"
args['enc_nhidden'] = 300
args['enc_nlayers'] = 1
args['enc_dropout'] = 0.2
args['dec_a_nhidden'] = 300
args['dec_a_nlayers'] = 1
args['dec_a_dropout'] = 0.2
args['dec_q_nhidden'] = 900
args['dec_q_nlayers'] = 2
args['dec_q_dropout'] = 0.3
args['nzqdim'] = 50
args['nza'] = 20
args['nzadim'] = 10
args['lambda_kl'] = 0.1
args['lambda_info'] = 1.0
args['device'] = torch.device('cpu')

In [ ]:
def get_squad_data_loader(tokenizer, file, shuffle, args):
    examples = read_squad_examples(file, is_training=True, debug=args['debug'])
    features = convert_examples_to_features_answer_id(examples,
                                                      tokenizer=tokenizer,
                                                      max_seq_length=args['max_c_len'],
                                                      max_query_length=args['max_q_len'],
                                                      max_ans_length=args['max_q_len'],
                                                      doc_stride=128,
                                                      is_training=True)

    all_c_ids = torch.tensor([f.c_ids for f in features], dtype=torch.long)
    all_q_ids = torch.tensor([f.q_ids for f in features], dtype=torch.long)
    all_tag_ids = torch.tensor([f.tag_ids for f in features], dtype=torch.long)
    all_a_ids = (all_tag_ids != 0).long()
    all_start_positions = torch.tensor([f.noq_start_position for f in features], dtype=torch.long)
    all_end_positions = torch.tensor([f.noq_end_position for f in features], dtype=torch.long)

    all_data = TensorDataset(all_c_ids, all_q_ids, all_a_ids, all_start_positions, all_end_positions)
    data_loader = DataLoader(all_data, args['batch_size'], shuffle=shuffle)

    return data_loader, examples, features

# **EVAL**

In [ ]:
from qgevalcap.eval import eval_qg
from squad_utils import evaluate, write_predictions
from utils import batch_to_device
import collections
import json

def to_string(index, tokenizer):
    tok_tokens = tokenizer.convert_ids_to_tokens(index)
    tok_text = " ".join(tok_tokens)

    # De-tokenize WordPieces that have been split off.
    tok_text = tok_text.replace("[PAD]", "")
    tok_text = tok_text.replace("[SEP]", "")
    tok_text = tok_text.replace("[CLS]", "")
    tok_text = tok_text.replace(" ##", "")
    tok_text = tok_text.replace("##", "")

    # Clean whitespace
    tok_text = tok_text.strip()
    tok_text = " ".join(tok_text.split())
    return tok_text

class Result(object):
    def __init__(self,
                 context,
                 real_question,
                 posterior_question,
                 prior_question,
                 real_answer,
                 posterior_answer,
                 prior_answer,
                 posterior_z_prob,
                 prior_z_prob):
        self.context = context
        self.real_question = real_question
        self.posterior_question = posterior_question
        self.prior_question = prior_question
        self.real_answer = real_answer
        self.posterior_answer = posterior_answer
        self.prior_answer = prior_answer
        self.posterior_z_prob = posterior_z_prob
        self.prior_z_prob = prior_z_prob


def eval_vae(epoch, args, trainer, eval_data):
    tokenizer = BertTokenizer.from_pretrained(args['bert_model'])
    RawResult = collections.namedtuple("RawResult",
                                       ["unique_id", "start_logits", "end_logits"])

    eval_loader, eval_examples, eval_features = eval_data

    all_results = []
    qa_results = []
    qg_results = {}
    res_dict = {}
    example_index = -1

    for batch in tqdm(eval_loader, desc="Eval iter", leave=False, position=4):
        c_ids, q_ids, a_ids, start, end = batch_to_device(batch, args['device'])
        batch_size = c_ids.size(0)
        batch_c_ids = c_ids.cpu().tolist()
        batch_q_ids = q_ids.cpu().tolist()
        batch_start = start.cpu().tolist()
        batch_end = end.cpu().tolist()

        batch_posterior_q_ids, \
        batch_posterior_start, batch_posterior_end, \
        posterior_z_prob = trainer.generate_posterior(c_ids, q_ids, a_ids)

        batch_start_logits, batch_end_logits \
        = trainer.generate_answer_logits(c_ids, q_ids, a_ids)

        batch_posterior_q_ids, \
        batch_posterior_start, batch_posterior_end = \
        batch_posterior_q_ids.cpu().tolist(), \
        batch_posterior_start.cpu().tolist(), batch_posterior_end.cpu().tolist()
        posterior_z_prob = posterior_z_prob.cpu()

        batch_prior_q_ids, \
        batch_prior_start, batch_prior_end, \
        prior_z_prob = trainer.generate_prior(c_ids)

        batch_prior_q_ids, \
        batch_prior_start, batch_prior_end = \
        batch_prior_q_ids.cpu().tolist(), \
        batch_prior_start.cpu().tolist(), batch_prior_end.cpu().tolist()
        prior_z_prob = prior_z_prob.cpu()

        for i in range(batch_size):
            example_index += 1
            start_logits = batch_start_logits[i].detach().cpu().tolist()
            end_logits = batch_end_logits[i].detach().cpu().tolist()
            eval_feature = eval_features[example_index]
            unique_id = int(eval_feature.unique_id)

            context = to_string(batch_c_ids[i], tokenizer)

            real_question = to_string(batch_q_ids[i], tokenizer)
            posterior_question = to_string(batch_posterior_q_ids[i], tokenizer)
            prior_question = to_string(batch_prior_q_ids[i], tokenizer)

            real_answer = to_string(batch_c_ids[i][batch_start[i]:(batch_end[i] + 1)], tokenizer)
            posterior_answer = to_string(batch_c_ids[i][batch_posterior_start[i]:(batch_posterior_end[i] + 1)], tokenizer)
            prior_answer = to_string(batch_c_ids[i][batch_prior_start[i]:(batch_prior_end[i] + 1)], tokenizer)

            all_results.append(Result(context=context,
                                      real_question=real_question,
                                      posterior_question=posterior_question,
                                      prior_question=prior_question,
                                      real_answer=real_answer,
                                      posterior_answer=posterior_answer,
                                      prior_answer=prior_answer,
                                      posterior_z_prob=posterior_z_prob[i],
                                      prior_z_prob=prior_z_prob[i]))

            qg_results[unique_id] = posterior_question
            res_dict[unique_id] = real_question
            qa_results.append(RawResult(unique_id=unique_id,
                                        start_logits=start_logits,
                                        end_logits=end_logits))

    output_prediction_file = os.path.join(args['model_dir'], "pred.json")
    write_predictions(eval_examples, eval_features, qa_results, n_best_size=20,
                      max_answer_length=30, do_lower_case=True,
                      output_prediction_file=output_prediction_file,
                      verbose_logging=False,
                      version_2_with_negative=False,
                      null_score_diff_threshold=0,
                      noq_position=True)

    with open(args['dev_dir']) as f:
        dataset_json = json.load(f)
        dataset = dataset_json["data"]
    with open(os.path.join(args['model_dir'], "pred.json")) as prediction_file:
        predictions = json.load(prediction_file)
    ret = evaluate(dataset, predictions)
    bleu = eval_qg(res_dict, qg_results)

    return ret, bleu, all_results

# **MODEL COMPONENTS**

In [ ]:
import numpy as np
import torch
import time
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch_scatter import scatter_max
from transformers import BertModel, BertTokenizer
import itertools

In [ ]:
# def scatter_max(src, index, out, dim=-1):
#     def make_slice(arr, dim, i):
#         slc = [slice(None)] * arr.ndim
#         slc[dim] = i
#         return slc

#     index = np.array(index.cpu())
#     if dim < 0:
#         # Not sure why scatter should accept dim < 0, but that is the behavior in PyTorch's scatter
#         dim = out.ndim + dim
#     idx_xsection_shape = index.shape[:dim] + index.shape[dim + 1:]
#     self_xsection_shape = out.shape[:dim] + out.shape[dim + 1:]

#     # We use index and dim parameters to create idx
#     # idx is in a form that can be used as a NumPy advanced index for scattering of src param. in out
#     idx = [[*np.indices(idx_xsection_shape).reshape(index.ndim - 1, -1),
#             index[tuple(make_slice(index, dim, i))].reshape(1, -1)[0]] for i in range(index.shape[dim])]
#     idx = list(np.concatenate(idx, axis=1))
#     idx.insert(dim, idx.pop())

#     src_idx = list(idx)
#     src_idx.pop(dim)
#     src_idx.insert(dim, np.repeat(np.arange(index.shape[dim]), np.prod(idx_xsection_shape)))
    
#     out[idx] = src[src_idx]

#     return out,1

def return_mask_lengths(ids):
    mask = torch.sign(ids).float()
    lengths = torch.sum(mask, 1)
    return mask, lengths


def cal_attn(query, memories, mask):
    mask = (1.0 - mask.float()) * -10000.0
    attn_logits = torch.matmul(query, memories.transpose(-1, -2).contiguous())
    attn_logits = attn_logits + mask
    attn_weights = F.softmax(attn_logits, dim=-1)
    attn_outputs = torch.matmul(attn_weights, memories)
    return attn_outputs, attn_logits


def gumbel_softmax(logits, tau=1, hard=False, eps=1e-20, dim=-1):
    # type: (Tensor, float, bool, float, int) -> Tensor

    gumbels = -(torch.empty_like(logits).exponential_() +
                eps).log()  # ~Gumbel(0,1)
    gumbels = (logits + gumbels) / tau  # ~Gumbel(logits,tau)
    y_soft = gumbels.softmax(dim)

    if hard:
        # Straight through.
        index = y_soft.max(dim, keepdim=True)[1]
        y_hard = torch.zeros_like(logits).scatter_(dim, index, 1.0)
        ret = y_hard - y_soft.detach() + y_soft
    else:
        # Re-parametrization trick.
        ret = y_soft
    return ret


class CategoricalKLLoss(nn.Module):
    def __init__(self):
        super(CategoricalKLLoss, self).__init__()

    def forward(self, P, Q):
        log_P = P.log()
        log_Q = Q.log()
        kl = (P * (log_P - log_Q)).sum(dim=-1).sum(dim=-1)
        return kl.mean(dim=0)


class GaussianKLLoss(nn.Module):
    def __init__(self):
        super(GaussianKLLoss, self).__init__()

    def forward(self, mu1, logvar1, mu2, logvar2):
        numerator = logvar1.exp() + torch.pow(mu1 - mu2, 2)
        fraction = torch.div(numerator, (logvar2.exp()))
        kl = 0.5 * torch.sum(logvar2 - logvar1 + fraction - 1, dim=1)
        return kl.mean(dim=0)

In [ ]:
class Embedding(nn.Module):
    def __init__(self, bert):
        super(Embedding, self).__init__()
        bert_embeddings = bert.embeddings
        self.word_embeddings = bert_embeddings.word_embeddings
        self.token_type_embeddings = bert_embeddings.token_type_embeddings
        self.position_embeddings = bert_embeddings.position_embeddings
        self.LayerNorm = bert_embeddings.LayerNorm
        self.dropout = bert_embeddings.dropout
        # print("word embeddings-er weight-er shape: ", self.word_embeddings.weight.shape)

    def forward(self, input_ids, token_type_ids=None, position_ids=None):
        if token_type_ids is None:
            token_type_ids = torch.zeros_like(input_ids)
        if position_ids is None:
            seq_length = input_ids.size(1)
            position_ids = torch.arange(
                seq_length, dtype=torch.long, device=input_ids.device)
            position_ids = position_ids.unsqueeze(0).expand_as(input_ids)

        words_embeddings = self.word_embeddings(input_ids)
        token_type_embeddings = self.token_type_embeddings(token_type_ids)
        position_embeddings = self.position_embeddings(position_ids)

        embeddings = words_embeddings + token_type_embeddings + position_embeddings
        embeddings = self.LayerNorm(embeddings)
        embeddings = self.dropout(embeddings)

        return embeddings


class ContextualizedEmbedding(nn.Module):
    def __init__(self, bert):
        super(ContextualizedEmbedding, self).__init__()
        
        self.embedding = bert.embeddings
        self.encoder = bert.encoder
        self.num_hidden_layers = bert.config.num_hidden_layers

    def forward(self, input_ids, attention_mask, token_type_ids=None):
        if token_type_ids is None:
            token_type_ids = torch.zeros_like(input_ids)

        seq_length = input_ids.size(1)
        position_ids = torch.arange(
            seq_length, dtype=torch.long, device=input_ids.device)
        position_ids = position_ids.unsqueeze(0).expand_as(input_ids)

        extended_attention_mask = attention_mask.unsqueeze(
            1).unsqueeze(2).float()
        extended_attention_mask = (1.0 - extended_attention_mask) * -10000.0
        head_mask = [None] * self.num_hidden_layers

        embedding_output = self.embedding(
            input_ids, position_ids=position_ids, token_type_ids=token_type_ids)
        encoder_outputs = self.encoder(embedding_output,
                                       extended_attention_mask,
                                       head_mask=head_mask)
        sequence_output = encoder_outputs[0]

        return sequence_output


In [ ]:
class CustomLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout, bidirectional=False):
        super(CustomLSTM, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.bidirectional = bidirectional
        self.dropout = nn.Dropout(dropout)
        if dropout > 0.0 and num_layers == 1:
            dropout = 0.0

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, dropout=dropout,
                            bidirectional=bidirectional, batch_first=True)

    def forward(self, inputs, input_lengths, state=None):
        _, total_length, _ = inputs.size()

        input_packed = pack_padded_sequence(inputs, input_lengths.cpu(),
                                            batch_first=True, enforce_sorted=False)

        self.lstm.flatten_parameters()
        output_packed, state = self.lstm(input_packed, state)

        output = pad_packed_sequence(
            output_packed, batch_first=True, total_length=total_length)[0]
        output = self.dropout(output)

        return output, state

In [ ]:
class PosteriorEncoder(nn.Module):
    def __init__(self, embedding, emsize,
                 nhidden, nlayers,
                 nzqdim, nza, nzadim,
                 dropout=0.0):
        super(PosteriorEncoder, self).__init__()

        self.embedding = embedding
        self.nhidden = nhidden
        self.nlayers = nlayers
        self.nzqdim = nzqdim
        self.nza = nza
        self.nzadim = nzadim

        self.encoder = CustomLSTM(input_size=emsize,
                                  hidden_size=nhidden,
                                  num_layers=nlayers,
                                  dropout=dropout,
                                  bidirectional=True)

        

        self.question_attention = nn.Linear(2 * nhidden, 2 * nhidden)
        self.context_attention = nn.Linear(2 * nhidden, 2 * nhidden)
        self.zq_attention = nn.Linear(nzqdim, 2 * nhidden)

        self.zq_linear = nn.Linear(4 * 2 * nhidden, 2 * nzqdim)
        self.za_linear = nn.Linear(nzqdim + 2 * 2 * nhidden, nza * nzadim)

    def forward(self, c_ids, q_ids, a_ids):
        c_mask, c_lengths = return_mask_lengths(c_ids)
        q_mask, q_lengths = return_mask_lengths(q_ids)

        # question enc
        q_embeddings = self.embedding(q_ids)
        q_hs, q_state = self.encoder(q_embeddings, q_lengths)
        q_h = q_state[0].view(self.nlayers, 2, -1, self.nhidden)[-1]
        q_h = q_h.transpose(0, 1).contiguous().view(-1, 2 * self.nhidden)

        # context enc
        c_embeddings = self.embedding(c_ids)
        c_hs, c_state = self.encoder(c_embeddings, c_lengths)
        c_h = c_state[0].view(self.nlayers, 2, -1, self.nhidden)[-1]
        c_h = c_h.transpose(0, 1).contiguous().view(-1, 2 * self.nhidden)

        # context and answer enc
        c_a_embeddings = self.embedding(c_ids, a_ids, None)
        c_a_hs, c_a_state = self.encoder(c_a_embeddings, c_lengths)
        c_a_h = c_a_state[0].view(self.nlayers, 2, -1, self.nhidden)[-1]
        c_a_h = c_a_h.transpose(0, 1).contiguous().view(-1, 2 * self.nhidden)

        # attetion q, c
        mask = c_mask.unsqueeze(1)
        c_attned_by_q, _ = cal_attn(self.question_attention(q_h).unsqueeze(1),
                                    c_hs,
                                    mask)
        c_attned_by_q = c_attned_by_q.squeeze(1)

        # attetion c, q
        mask = q_mask.unsqueeze(1)
        q_attned_by_c, _ = cal_attn(self.context_attention(c_h).unsqueeze(1),
                                    q_hs,
                                    mask)
        q_attned_by_c = q_attned_by_c.squeeze(1)

        h = torch.cat([q_h, q_attned_by_c, c_h, c_attned_by_q], dim=-1)

        zq_mu, zq_logvar = torch.split(self.zq_linear(h), self.nzqdim, dim=1)
        zq = zq_mu + torch.randn_like(zq_mu) * torch.exp(0.5 * zq_logvar)

        # attention zq, c_a
        mask = c_mask.unsqueeze(1)
        c_a_attned_by_zq, _ = cal_attn(self.zq_attention(zq).unsqueeze(1),
                                       c_a_hs,
                                       mask)
        c_a_attned_by_zq = c_a_attned_by_zq.squeeze(1)

        h = torch.cat([zq, c_a_attned_by_zq, c_a_h], dim=-1)

        za_logits = self.za_linear(h).view(-1, self.nza, self.nzadim)
        za_prob = F.softmax(za_logits, dim=-1)
        za = gumbel_softmax(za_logits, hard=True)

        return zq_mu, zq_logvar, zq, za_prob, za

In [ ]:
class PriorEncoder(nn.Module):
    def __init__(self, embedding, emsize,
                 nhidden, nlayers,
                 nzqdim, nza, nzadim,
                 dropout=0):
        super(PriorEncoder, self).__init__()

        self.embedding = embedding
        self.nhidden = nhidden
        self.nlayers = nlayers
        self.nzqdim = nzqdim
        self.nza = nza
        self.nzadim = nzadim

        self.context_encoder = CustomLSTM(input_size=emsize,
                                          hidden_size=nhidden,
                                          num_layers=nlayers,
                                          dropout=dropout,
                                          bidirectional=True)

        self.zq_attention = nn.Linear(nzqdim, 2 * nhidden)

        self.zq_linear = nn.Linear(2 * nhidden, 2 * nzqdim)
        self.za_linear = nn.Linear(nzqdim + 2 * 2 * nhidden, nza * nzadim)

    def forward(self, c_ids):
        c_mask, c_lengths = return_mask_lengths(c_ids)

        c_embeddings = self.embedding(c_ids)
        c_hs, c_state = self.context_encoder(c_embeddings, c_lengths)
        c_h = c_state[0].view(self.nlayers, 2, -1, self.nhidden)[-1]
        c_h = c_h.transpose(0, 1).contiguous().view(-1, 2 * self.nhidden)

        zq_mu, zq_logvar = torch.split(self.zq_linear(c_h), self.nzqdim, dim=1)
        zq = zq_mu + torch.randn_like(zq_mu)*torch.exp(0.5*zq_logvar)

        mask = c_mask.unsqueeze(1)
        c_attned_by_zq, _ = cal_attn(self.zq_attention(zq).unsqueeze(1),
                                     c_hs,
                                     mask)
        c_attned_by_zq = c_attned_by_zq.squeeze(1)

        h = torch.cat([zq, c_attned_by_zq, c_h], dim=-1)

        za_logits = self.za_linear(h).view(-1, self.nza, self.nzadim)
        za_prob = F.softmax(za_logits, dim=-1)
        za = gumbel_softmax(za_logits, hard=True)

        return zq_mu, zq_logvar, zq, za_prob, za

    def interpolation(self, c_ids, zq):

        c_mask, c_lengths = return_mask_lengths(c_ids)

        c_embeddings = self.embedding(c_ids)
        c_hs, c_state = self.context_encoder(c_embeddings, c_lengths)
        c_h = c_state[0].view(self.nlayers, 2, -1, self.nhidden)[-1]
        c_h = c_h.transpose(0, 1).contiguous().view(-1, 2 * self.nhidden)

        mask = c_mask.unsqueeze(1)
        c_attned_by_zq, _ = cal_attn(
            self.zq_attention(zq).unsqueeze(1), c_hs, mask)
        c_attned_by_zq = c_attned_by_zq.squeeze(1)

        h = torch.cat([zq, c_attned_by_zq, c_h], dim=-1)

        za_logits = self.za_linear(h).view(-1, self.nza, self.nzadim)
        za = gumbel_softmax(za_logits, hard=True)

        return za

In [ ]:
class AnswerDecoder(nn.Module):
    def __init__(self, embedding, emsize,
                 nhidden, nlayers,
                 dropout=0.0):
        super(AnswerDecoder, self).__init__()

        self.embedding = embedding

        self.context_lstm = CustomLSTM(input_size=4 * emsize,
                                       hidden_size=nhidden,
                                       num_layers=nlayers,
                                       dropout=dropout,
                                       bidirectional=True)

        self.start_linear = nn.Linear(2 * nhidden, 1)
        self.end_linear = nn.Linear(2 * nhidden, 1)
        self.ls = nn.LogSoftmax(dim=1)

    def forward(self, init_state, c_ids):
        _, max_c_len = c_ids.size()
        c_mask, c_lengths = return_mask_lengths(c_ids)

        H = self.embedding(c_ids, c_mask)
        U = init_state.unsqueeze(1).repeat(1, max_c_len, 1)
        G = torch.cat([H, U, H * U, torch.abs(H - U)], dim=-1)
        M, _ = self.context_lstm(G, c_lengths)

        start_logits = self.start_linear(M).squeeze(-1)
        end_logits = self.end_linear(M).squeeze(-1)

        start_end_mask = (c_mask == 0)
        masked_start_logits = start_logits.masked_fill(
            start_end_mask, -10000.0)
        masked_end_logits = end_logits.masked_fill(start_end_mask, -10000.0)

        return masked_start_logits, masked_end_logits

    def generate(self, init_state, c_ids):
        start_logits, end_logits = self.forward(init_state, c_ids)
        c_mask, _ = return_mask_lengths(c_ids)
        batch_size, max_c_len = c_ids.size()

        mask = torch.matmul(c_mask.unsqueeze(2).float(),
                            c_mask.unsqueeze(1).float())
        mask = torch.triu(mask) == 0
        score = (self.ls(start_logits).unsqueeze(2)
                 + self.ls(end_logits).unsqueeze(1))
        score = score.masked_fill(mask, -10000.0)
        score, start_positions = score.max(dim=1)
        score, end_positions = score.max(dim=1)
        start_positions = torch.gather(start_positions,
                                       1,
                                       end_positions.view(-1, 1)).squeeze(1)

        idxes = torch.arange(0, max_c_len, out=torch.LongTensor(max_c_len))
        idxes = idxes.unsqueeze(0).to(
            start_logits.device).repeat(batch_size, 1)

        start_positions = start_positions.unsqueeze(1)
        start_mask = (idxes >= start_positions).long()
        end_positions = end_positions.unsqueeze(1)
        end_mask = (idxes <= end_positions).long()
        a_ids = start_mask + end_mask - 1

        return a_ids, start_positions.squeeze(1), end_positions.squeeze(1)

In [ ]:
class ContextEncoderforQG(nn.Module):
    def __init__(self, embedding, emsize,
                 nhidden, nlayers,
                 dropout=0.0):
        super(ContextEncoderforQG, self).__init__()
        self.embedding = embedding
        self.context_lstm = CustomLSTM(input_size=emsize,
                                       hidden_size=nhidden,
                                       num_layers=nlayers,
                                       dropout=dropout,
                                       bidirectional=True)
        self.context_linear = nn.Linear(2 * nhidden, 2 * nhidden)
        self.fusion = nn.Linear(4 * nhidden, 2 * nhidden, bias=False)
        self.gate = nn.Linear(4 * nhidden, 2 * nhidden, bias=False)

    def forward(self, c_ids, a_ids):
        c_mask, c_lengths = return_mask_lengths(c_ids)
        c_embeddings = self.embedding(c_ids, c_mask, a_ids)
        c_outputs, _ = self.context_lstm(c_embeddings, c_lengths)
        # attention
        mask = torch.matmul(c_mask.unsqueeze(2), c_mask.unsqueeze(1))
        c_attned_by_c, _ = cal_attn(self.context_linear(c_outputs),
                                    c_outputs,
                                    mask)
        c_concat = torch.cat([c_outputs, c_attned_by_c], dim=2)
        c_fused = self.fusion(c_concat).tanh()
        c_gate = self.gate(c_concat).sigmoid()
        c_outputs = c_gate * c_fused + (1 - c_gate) * c_outputs
        return c_outputs


class QuestionDecoder(nn.Module):
    def __init__(self, sos_id, eos_id,
                 embedding, contextualized_embedding, emsize,
                 nhidden, ntokens, nlayers,
                 dropout=0.0,
                 max_q_len=64):
        super(QuestionDecoder, self).__init__()

        self.sos_id = sos_id
        self.eos_id = eos_id
        self.emsize = emsize
        self.embedding = embedding
        self.nhidden = nhidden
        self.ntokens = ntokens
        self.nlayers = nlayers
        # this max_len include sos eos
        self.max_q_len = max_q_len

        self.context_lstm = ContextEncoderforQG(contextualized_embedding, emsize,
                                                nhidden // 2, nlayers, dropout)

        self.question_lstm = CustomLSTM(input_size=emsize,
                                        hidden_size=nhidden,
                                        num_layers=nlayers,
                                        dropout=dropout,
                                        bidirectional=False)

        self.question_linear = nn.Linear(nhidden, nhidden)

        self.concat_linear = nn.Sequential(nn.Linear(2*nhidden, 2*nhidden),
                                           nn.Dropout(dropout),
                                           nn.Linear(2*nhidden, 2*emsize))
        
        self.logit_linear = nn.Linear(emsize, ntokens, bias=False)

        # fix output word matrix
        self.logit_linear.weight = embedding.word_embeddings.weight
        for param in self.logit_linear.parameters():
            param.requires_grad = False

        self.discriminator = nn.Bilinear(emsize, nhidden, 1)

    def postprocess(self, q_ids):
        eos_mask = q_ids == self.eos_id
        no_eos_idx_sum = (eos_mask.sum(dim=1) == 0).long() * \
            (self.max_q_len - 1)
        eos_mask = eos_mask.cpu().numpy()
        q_lengths = np.argmax(eos_mask, axis=1) + 1
        q_lengths = torch.tensor(q_lengths).to(
            q_ids.device).long() + no_eos_idx_sum
        batch_size, max_len = q_ids.size()
        idxes = torch.arange(0, max_len).to(q_ids.device)
        idxes = idxes.unsqueeze(0).repeat(batch_size, 1)
        q_mask = (idxes < q_lengths.unsqueeze(1))
        q_ids = q_ids.long() * q_mask.long()
        return q_ids

    def forward(self, init_state, c_ids, q_ids, a_ids):
        batch_size, max_q_len = q_ids.size()

        c_outputs = self.context_lstm(c_ids, a_ids)

        c_mask, _ = return_mask_lengths(c_ids)
        q_mask, q_lengths = return_mask_lengths(q_ids)

        # question dec
        q_embeddings = self.embedding(q_ids)
        q_outputs, _ = self.question_lstm(q_embeddings, q_lengths, init_state)

        # attention
        mask = torch.matmul(q_mask.unsqueeze(2), c_mask.unsqueeze(1))
        c_attned_by_q, attn_logits = cal_attn(self.question_linear(q_outputs),
                                              c_outputs,
                                              mask)

        # gen logits
        q_concated = torch.cat([q_outputs, c_attned_by_q], dim=2)
        q_concated = self.concat_linear(q_concated)
        q_maxouted, _ = q_concated.view(
            batch_size, max_q_len, self.emsize, 2).max(dim=-1)
        gen_logits = self.logit_linear(q_maxouted)

        # copy logits
        bq = batch_size * max_q_len
        c_ids = c_ids.unsqueeze(1).repeat(
            1, max_q_len, 1).view(bq, -1).contiguous()
        attn_logits = attn_logits.view(bq, -1).contiguous()
        copy_logits = torch.zeros(bq, self.ntokens).to(c_ids.device)
        copy_logits = copy_logits - 10000.0
        # print(self.emsize)
        t0 = time.time()
        # print(attn_logits.shape)
        # print(c_ids.shape)
        # print(copy_logits.shape)
        copy_logits, _ = scatter_max(attn_logits, c_ids, out=copy_logits)
        # print('scatter_max time: ', time.time()-t0)
        copy_logits = copy_logits.masked_fill(copy_logits == -10000.0, 0)
        copy_logits = copy_logits.view(batch_size, max_q_len, -1).contiguous()

        logits = gen_logits + copy_logits

        # mutual information btw answer and question
        a_emb = c_outputs * a_ids.float().unsqueeze(2)
        a_mean_emb = torch.sum(a_emb, 1) / a_ids.sum(1).unsqueeze(1).float()
        fake_a_mean_emb = torch.cat([a_mean_emb[-1].unsqueeze(0),
                                     a_mean_emb[:-1]], dim=0)

        q_emb = q_maxouted * q_mask.unsqueeze(2)
        q_mean_emb = torch.sum(q_emb, 1) / q_lengths.unsqueeze(1).float()
        fake_q_mean_emb = torch.cat([q_mean_emb[-1].unsqueeze(0),
                                     q_mean_emb[:-1]], dim=0)

        bce_loss = nn.BCEWithLogitsLoss()
        true_logits = self.discriminator(q_mean_emb, a_mean_emb)
        true_labels = torch.ones_like(true_logits)

        fake_a_logits = self.discriminator(q_mean_emb, fake_a_mean_emb)
        fake_q_logits = self.discriminator(fake_q_mean_emb, a_mean_emb)
        fake_logits = torch.cat([fake_a_logits, fake_q_logits], dim=0)
        fake_labels = torch.zeros_like(fake_logits)

        true_loss = bce_loss(true_logits, true_labels)
        fake_loss = 0.5 * bce_loss(fake_logits, fake_labels)
        loss_info = 0.5 * (true_loss + fake_loss)

        return logits, loss_info

    def generate(self, init_state, c_ids, a_ids):
        c_mask, _ = return_mask_lengths(c_ids)
        c_outputs = self.context_lstm(c_ids, a_ids)

        batch_size = c_ids.size(0)

        q_ids = torch.LongTensor([self.sos_id] * batch_size).unsqueeze(1)
        q_ids = q_ids.to(c_ids.device)
        token_type_ids = torch.zeros_like(q_ids)
        position_ids = torch.zeros_like(q_ids)
        q_embeddings = self.embedding(q_ids, token_type_ids, position_ids)

        state = init_state

        # unroll
        all_q_ids = list()
        all_q_ids.append(q_ids)
        for _ in range(self.max_q_len - 1):
            position_ids = position_ids + 1
            q_outputs, state = self.question_lstm.lstm(q_embeddings, state)

            # attention
            mask = c_mask.unsqueeze(1)
            c_attned_by_q, attn_logits = cal_attn(self.question_linear(q_outputs),
                                                  c_outputs,
                                                  mask)

            # gen logits
            q_concated = torch.cat([q_outputs, c_attned_by_q], dim=2)
            q_concated = self.concat_linear(q_concated)
            q_maxouted, _ = q_concated.view(
                batch_size, 1, self.emsize, 2).max(dim=-1)
            gen_logits = self.logit_linear(q_maxouted)

            # copy logits
            attn_logits = attn_logits.squeeze(1)
            copy_logits = torch.zeros(
                batch_size, self.ntokens).to(c_ids.device)
            copy_logits = copy_logits - 10000.0
            copy_logits, _ = scatter_max(attn_logits, c_ids, out=copy_logits)
            copy_logits = copy_logits.masked_fill(copy_logits == -10000.0, 0)

            logits = gen_logits + copy_logits.unsqueeze(1)

            q_ids = torch.argmax(logits, 2)
            all_q_ids.append(q_ids)

            q_embeddings = self.embedding(q_ids, token_type_ids, position_ids)

        q_ids = torch.cat(all_q_ids, 1)
        q_ids = self.postprocess(q_ids)

        return q_ids

    def sample(self, init_state, c_ids, a_ids):
        c_mask, c_lengths = return_mask_lengths(c_ids)
        c_outputs = self.context_lstm(c_ids, a_ids)

        batch_size = c_ids.size(0)

        q_ids = torch.LongTensor([self.sos_id] * batch_size).unsqueeze(1)
        q_ids = q_ids.to(c_ids.device)
        token_type_ids = torch.zeros_like(q_ids)
        position_ids = torch.zeros_like(q_ids)
        q_embeddings = self.embedding(q_ids, token_type_ids, position_ids)

        state = init_state

        # unroll
        all_q_ids = list()
        all_q_ids.append(q_ids)
        for _ in range(self.max_q_len - 1):
            position_ids = position_ids + 1
            q_outputs, state = self.question_lstm.lstm(q_embeddings, state)

            # attention
            mask = c_mask.unsqueeze(1)
            c_attned_by_q, attn_logits = cal_attn(self.question_linear(q_outputs),
                                                  c_outputs,
                                                  mask)

            # gen logits
            q_concated = torch.cat([q_outputs, c_attned_by_q], dim=2)
            q_concated = self.concat_linear(q_concated)
            q_maxouted, _ = q_concated.view(batch_size, 1, self.emsize, 2).max(dim=-1)
            gen_logits = self.logit_linear(q_maxouted)

            # copy logits
            attn_logits = attn_logits.squeeze(1)
            copy_logits = torch.zeros(batch_size, self.ntokens).to(c_ids.device)
            copy_logits = copy_logits - 10000.0
            copy_logits, _ = scatter_max(attn_logits, c_ids, out=copy_logits)
            copy_logits = copy_logits.masked_fill(copy_logits == -10000.0, 0)

            logits = gen_logits + copy_logits.unsqueeze(1)
            logits = logits.squeeze(1)
            logits = self.top_k_top_p_filtering(logits, 2, top_p=0.8)
            probs = F.softmax(logits, dim=-1)
            q_ids = torch.multinomial(probs, num_samples=1)  # [b,1]
            all_q_ids.append(q_ids)

            q_embeddings = self.embedding(q_ids, token_type_ids, position_ids)

        q_ids = torch.cat(all_q_ids, 1)
        q_ids = self.postprocess(q_ids)

        return q_ids

# **MODEL**

In [ ]:
class DiscreteVAE(nn.Module):
    def __init__(self, args):
        super(DiscreteVAE, self).__init__()
        bert_model = args['bert_model']
        tokenizer = BertTokenizer.from_pretrained(bert_model)
        bert = BertModel.from_pretrained(bert_model)
        padding_idx = tokenizer.vocab['[PAD]']
        sos_id = tokenizer.vocab['[CLS]']
        eos_id = tokenizer.vocab['[SEP]']
        # ntokens = len(tokenizer.vocab)
        ntokens = bert.embeddings.word_embeddings.weight.shape[0]
        if "large" in bert_model:
            emsize = 1024
        elif "base" in bert_model:
            emsize = 768
        elif "small" in bert_model:
            emsize = 512
        else:
            emsize = 768

        enc_nhidden = args['enc_nhidden']
        enc_nlayers = args['enc_nlayers']
        enc_dropout = args['enc_dropout']
        dec_a_nhidden = args['dec_a_nhidden']
        dec_a_nlayers = args['dec_a_nlayers']
        dec_a_dropout = args['dec_a_dropout']
        self.dec_q_nhidden = dec_q_nhidden = args['dec_q_nhidden']
        self.dec_q_nlayers = dec_q_nlayers = args['dec_q_nlayers']
        dec_q_dropout = args['dec_q_dropout']
        self.nzqdim = nzqdim = args['nzqdim']
        self.nza = nza = args['nza']
        self.nzadim = nzadim = args['nzadim']

        self.lambda_kl = args['lambda_kl']
        self.lambda_info = args['lambda_info']

        max_q_len = args['max_q_len']

        embedding = Embedding(bert)
        contextualized_embedding = ContextualizedEmbedding(bert)
        # freeze embedding
        for param in embedding.parameters():
            param.requires_grad = False
        for param in contextualized_embedding.parameters():
            param.requires_grad = False

        self.posterior_encoder = PosteriorEncoder(embedding, emsize,
                                                  enc_nhidden, enc_nlayers,
                                                  nzqdim, nza, nzadim,
                                                  enc_dropout)

        self.prior_encoder = PriorEncoder(embedding, emsize,
                                          enc_nhidden, enc_nlayers,
                                          nzqdim, nza, nzadim, enc_dropout)

        self.answer_decoder = AnswerDecoder(contextualized_embedding, emsize,
                                            dec_a_nhidden, dec_a_nlayers,
                                            dec_a_dropout)

        self.question_decoder = QuestionDecoder(sos_id, eos_id,
                                                embedding, contextualized_embedding, emsize,
                                                dec_q_nhidden, ntokens, dec_q_nlayers,
                                                dec_q_dropout,
                                                max_q_len)

        self.q_h_linear = nn.Linear(nzqdim, dec_q_nlayers * dec_q_nhidden)
        self.q_c_linear = nn.Linear(nzqdim, dec_q_nlayers * dec_q_nhidden)
        self.a_linear = nn.Linear(nza * nzadim, emsize, False)

        self.q_rec_criterion = nn.CrossEntropyLoss(ignore_index=padding_idx)
        self.gaussian_kl_criterion = GaussianKLLoss()
        self.categorical_kl_criterion = CategoricalKLLoss()

    def return_init_state(self, zq, za):

        q_init_h = self.q_h_linear(zq)
        q_init_c = self.q_c_linear(zq)
        q_init_h = q_init_h.view(-1, self.dec_q_nlayers,
                                 self.dec_q_nhidden).transpose(0, 1).contiguous()
        q_init_c = q_init_c.view(-1, self.dec_q_nlayers,
                                 self.dec_q_nhidden).transpose(0, 1).contiguous()
        q_init_state = (q_init_h, q_init_c)

        za_flatten = za.view(-1, self.nza * self.nzadim)
        a_init_state = self.a_linear(za_flatten)

        return q_init_state, a_init_state


    def forward(self, c_ids, q_ids, a_ids, start_positions, end_positions):

        t0 = time.time()
        posterior_zq_mu, posterior_zq_logvar, posterior_zq, \
            posterior_za_prob, posterior_za \
            = self.posterior_encoder(c_ids, q_ids, a_ids)

        #print('posterior encoder time: ', time.time()-t0)
        t0 = time.time()

        prior_zq_mu, prior_zq_logvar, _, \
            prior_za_prob, _ \
            = self.prior_encoder(c_ids)

        #print('prior encoder time: ', time.time()-t0)
        t0 = time.time()

        q_init_state, a_init_state = self.return_init_state(
            posterior_zq, posterior_za)
        
        # answer decoding
        start_logits, end_logits = self.answer_decoder(a_init_state, c_ids)
        # question decoding

        #print('answer decoder time: ', time.time()-t0)
        t0 = time.time()

        q_logits, loss_info = self.question_decoder(
            q_init_state, c_ids, q_ids, a_ids)
        
        #print('question_decoder encoder time: ', time.time()-t0)

        # q rec loss
        loss_q_rec = self.q_rec_criterion(q_logits[:, :-1, :].transpose(1, 2).contiguous(),
                                          q_ids[:, 1:])

        # a rec loss
        max_c_len = c_ids.size(1)
        a_rec_criterion = nn.CrossEntropyLoss(ignore_index=max_c_len)
        start_positions.clamp_(0, max_c_len)
        end_positions.clamp_(0, max_c_len)
        loss_start_a_rec = a_rec_criterion(start_logits, start_positions)
        loss_end_a_rec = a_rec_criterion(end_logits, end_positions)
        loss_a_rec = 0.5 * (loss_start_a_rec + loss_end_a_rec)

        # kl loss
        loss_zq_kl = self.gaussian_kl_criterion(posterior_zq_mu,
                                                posterior_zq_logvar,
                                                prior_zq_mu,
                                                prior_zq_logvar)

        loss_za_kl = self.categorical_kl_criterion(posterior_za_prob,
                                                   prior_za_prob)

        loss_kl = self.lambda_kl * (loss_zq_kl + loss_za_kl)
        loss_info = self.lambda_info * loss_info

        loss = loss_q_rec + loss_a_rec + loss_kl + loss_info

        return loss, \
            loss_q_rec, loss_a_rec, \
            loss_zq_kl, loss_za_kl, \
            loss_info

    def generate(self, zq, za, c_ids):
        q_init_state, a_init_state = self.return_init_state(zq, za)

        a_ids, start_positions, end_positions = self.answer_decoder.generate(
            a_init_state, c_ids)

        q_ids = self.question_decoder.generate(q_init_state, c_ids, a_ids)

        return q_ids, start_positions, end_positions

    def return_answer_logits(self, zq, za, c_ids):
        _, a_init_state = self.return_init_state(zq, za)

        start_logits, end_logits = self.answer_decoder(a_init_state, c_ids)

        return start_logits, end_logits


# **TRAINER**

In [ ]:
#!cp -r '/content/drive/MyDrive/Thesis/HCVAE/saveHCVAEBANGLA3' '/content/Info-HCVAE/save'
#!rm -r '/content/Info-HCVAE/save'

In [ ]:
import torch
import torch.nn as nn
import os

# from models import DiscreteVAE, return_mask_lengths


class VAETrainer(object):
    def __init__(self, args):
        self.args = args
        self.clip = args['clip']
        self.device = args['device']

        if os.path.isfile(args['load_checkpoint']):
            print("Loading from checkpoint...")
            checkpoint = torch.load(args['load_checkpoint'], map_location="cpu")
            self.vae = DiscreteVAE(checkpoint["args"]).to(self.device)
            self.vae.load_state_dict(checkpoint["state_dict"])
        else:
            self.vae = DiscreteVAE(args).to(self.device)

        params = filter(lambda p: p.requires_grad, self.vae.parameters())
        self.optimizer = torch.optim.Adam(params, lr=args['lr'])

        self.loss_q_rec = 0
        self.loss_a_rec = 0
        self.loss_zq_kl = 0
        self.loss_za_kl = 0
        self.loss_info = 0

    def train(self, c_ids, q_ids, a_ids, start_positions, end_positions):
        self.vae = self.vae.train()
        t0 = time.time()
        # Forward
        loss, \
        loss_q_rec, loss_a_rec, \
        loss_zq_kl, loss_za_kl, \
        loss_info \
        = self.vae(c_ids, q_ids, a_ids, start_positions, end_positions)
        #print('forward time: ', time.time()-t0)
        # Backward
        self.optimizer.zero_grad()
        loss.backward()

        # Step
        self.optimizer.step()

        self.loss_q_rec = loss_q_rec.item()
        self.loss_a_rec = loss_a_rec.item()
        self.loss_zq_kl = loss_zq_kl.item()
        self.loss_za_kl = loss_za_kl.item()
        self.loss_info = loss_info.item()

    def generate_posterior(self, c_ids, q_ids, a_ids):
        self.vae = self.vae.eval()
        with torch.no_grad():
            _, _, zq, _, za = self.vae.posterior_encoder(c_ids, q_ids, a_ids)
            q_ids, start_positions, end_positions = self.vae.generate(zq, za, c_ids)
        return q_ids, start_positions, end_positions, zq

    def generate_answer_logits(self, c_ids, q_ids, a_ids):
        self.vae = self.vae.eval()
        with torch.no_grad():
            _, _, zq, _, za = self.vae.posterior_encoder(c_ids, q_ids, a_ids)
            start_logits, end_logits = self.vae.return_answer_logits(zq, za, c_ids)
        return start_logits, end_logits

    def generate_prior(self, c_ids):
        self.vae = self.vae.eval()
        with torch.no_grad():
            _, _, zq, _, za = self.vae.prior_encoder(c_ids)
            q_ids, start_positions, end_positions = self.vae.generate(zq, za, c_ids)
        return q_ids, start_positions, end_positions, zq    

    def save(self, filename):
        args = {
            'state_dict': self.vae.state_dict(),
            'args': self.args
        }
        torch.save(args, filename)

In [ ]:
os.makedirs(args['model_dir'], exist_ok=True)
args['model_dir'] = os.path.abspath(args['model_dir'])

random.seed(args['seed'])
np.random.seed(args['seed'])
torch.manual_seed(args['seed'])
torch.cuda.manual_seed(args['seed'])

tokenizer = BertTokenizer.from_pretrained(args['bert_model'])
train_loader, _, _ = get_squad_data_loader(tokenizer, args['train_dir'],
                                        shuffle=True, args=args)
eval_data = get_squad_data_loader(tokenizer, args['dev_dir'],
                                    shuffle=False, args=args)


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

100%|██████████| 10570/10570 [02:16<00:00, 77.53it/s]


In [ ]:
args['device'] = torch.cuda.current_device()

if torch.cuda.is_available():
    args['device'] = torch.device("cuda")

trainer = VAETrainer(args)

Loading from checkpoint...


Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
loss_log1 = tqdm(total=0, bar_format='{desc}', position=2)
loss_log2 = tqdm(total=0, bar_format='{desc}', position=3)
eval_log = tqdm(total=0, bar_format='{desc}', position=5)
best_eval_log = tqdm(total=0, bar_format='{desc}', position=6)

print("MODEL DIR: " + args['model_dir'])

best_bleu, best_em, best_f1 = 0.0, 0.0, 0.0
for epoch in trange(int(args['epochs']), desc="Epoch", position=0):
    for batch in tqdm(train_loader, desc="Train iter", leave=False, position=1):
        c_ids, q_ids, a_ids, start_positions, end_positions \
        = batch_to_device(batch, args['device'])
        trainer.train(c_ids, q_ids, a_ids, start_positions, end_positions)
        
        str1 = 'Q REC : {:06.4f} A REC : {:06.4f}'
        str2 = 'ZQ KL : {:06.4f} ZA KL : {:06.4f} INFO : {:06.4f}'
        str1 = str1.format(float(trainer.loss_q_rec), float(trainer.loss_a_rec))
        str2 = str2.format(float(trainer.loss_zq_kl), float(trainer.loss_za_kl), float(trainer.loss_info))
        loss_log1.set_description_str(str1)
        loss_log2.set_description_str(str2)

    
    metric_dict, bleu, _ = eval_vae(epoch, args, trainer, eval_data)
    f1 = metric_dict["f1"]
    em = metric_dict["exact_match"]
    bleu = bleu * 100
    _str = '{}-th Epochs BLEU : {:02.2f} EM : {:02.2f} F1 : {:02.2f}'
    _str = _str.format(epoch, bleu, em, f1)
    eval_log.set_description_str(_str)
    if em > best_em:
        best_em = em
    if f1 > best_f1:
        best_f1 = f1
        print("Saving new best f1 model...")
        trainer.save(os.path.join(args['model_dir'], "best_f1_model.pt"))
    if bleu > best_bleu:
        best_bleu = bleu
        print("Saving new best bleu model...")
        trainer.save(os.path.join(args['model_dir'], "best_bleu_model.pt"))
    
    _str = 'BEST BLEU : {:02.2f} EM : {:02.2f} F1 : {:02.2f}'
    _str = _str.format(best_bleu, best_em, best_f1)
    best_eval_log.set_description_str(_str)

MODEL DIR: /content/drive/MyDrive/Thesis/HCVAE/save_english/save/vae-checkpoint


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Train iter:   0%|          | 0/2750 [00:00<?, ?it/s]

Eval iter:   0%|          | 0/334 [00:00<?, ?it/s]

Saving new best f1 model...
Saving new best bleu model...


...

Eval iter:   0%|          | 0/334 [00:00<?, ?it/s]

Saving new best f1 model...


Train iter:   0%|          | 0/2750 [00:00<?, ?it/s]

Eval iter:   0%|          | 0/334 [00:00<?, ?it/s]

Saving new best f1 model...


# Save Model Location

In [ ]:
#!cp -r '/content/Info-HCVAE/save' '/content/drive/MyDrive/saveHCVAEBANGLA9org'

# **Evaluation**

In [ ]:
!mkdir "/content/Info-HCVAE/save"
!mkdir "/content/Info-HCVAE/save/vae-checkpoint"

In [ ]:
cp -r "/content/drive/MyDrive/Thesis/HCVAE/save_english/save/vae-checkpoint/best_f1_model.pt" "/content/Info-HCVAE/save/vae-checkpoint/best_f1_model.pt"

In [ ]:
args['max_c_len'] = 384-64
args['max_q_len'] = 0
args['squad'] = True
args['data_file'] = "../data/squad/train-v1.1.json"
args['checkpoint'] = "../save/vae-checkpoint/best_f1_model.pt" # Changed this to f1 model
args['output_file'] = "../data/synthetic_data/1.0_squad_10x_features.pkl"
args['ratio'] = 1.0
args['k'] = 10

In [ ]:
import pickle

import torch
from transformers import BertTokenizer
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
import os
from squad_utils import (InputFeatures, convert_examples_to_harv_features,
                         read_examples, read_squad_examples)


def return_mask_lengths(ids):
    mask = torch.sign(ids).float()
    lengths = torch.sum(mask, 1).long()
    return mask, lengths


def post_process(q_ids, start_positions, end_positions, c_ids, total_max_len=384):
    """
       concatenate question and context for BERT QA model:
       [CLS] Question [SEP] Context [SEP]
    """
    batch_size = q_ids.size(0)
    # exclude CLS token in c_ids
    c_ids = c_ids[:, 1:]
    start_positions = start_positions - 1
    end_positions = end_positions - 1

    _, q_lengths = return_mask_lengths(q_ids)
    _, c_lengths = return_mask_lengths(c_ids)

    all_input_ids = []
    all_seg_ids = []
    for i in range(batch_size):
        q_length = q_lengths[i]
        c_length = c_lengths[i]
        q = q_ids[i, :q_length]  # exclude pad tokens
        c = c_ids[i, :c_length]  # exclude pad tokens

        # input ids
        pads = torch.zeros((total_max_len - q_length - c_length), device=q_ids.device, dtype=torch.long)
        input_ids = torch.cat([q, c, pads], dim=0)
        all_input_ids.append(input_ids)

        # segment ids
        zeros = torch.zeros_like(q)
        ones = torch.ones_like(c)
        seg_ids = torch.cat([zeros, ones, pads], dim=0)
        all_seg_ids.append(seg_ids)

        start_positions[i] = start_positions[i] + q_length
        end_positions[i] = end_positions[i] + q_length

    all_input_ids = torch.stack(all_input_ids, dim=0)
    all_seg_ids = torch.stack(all_seg_ids, dim=0)
    all_input_mask = (all_input_ids != 0).byte()

    return all_input_ids, all_seg_ids, all_input_mask, start_positions, end_positions



In [ ]:
# tokenizer = BertTokenizer.from_pretrained(args['bert_model'])
# sos_id = tokenizer.vocab['[CLS]']
# eos_id = tokenizer.vocab['[SEP]']
# print(sos_id)
# print(eos_id)

101
102


In [ ]:
tokenizer = BertTokenizer.from_pretrained(args['bert_model'])
args['tokenizer'] = tokenizer
args['debug'] = True
args['squad'] = True
device = torch.cuda.current_device()
checkpoint = torch.load(args['checkpoint'], map_location="cpu")
vae = DiscreteVAE(checkpoint["args"])
vae.load_state_dict(checkpoint["state_dict"])
vae.eval()
vae = vae.to(device)

if args['squad']:
    examples = read_squad_examples(args['data_file'], is_training=True, debug=args['debug'])
    features = convert_examples_to_harv_features(examples,
                                                tokenizer=tokenizer,
                                                max_seq_length=args['max_c_len'],
                                                max_query_length=args['max_q_len'],
                                                doc_stride=128,
                                                is_training=True)
else:
    examples = read_examples(args['data_file'], is_training=True, debug=args['debug'])
    features = convert_examples_to_harv_features(examples,
                                                tokenizer=tokenizer,
                                                max_seq_length=args['max_c_len'],
                                                max_query_length=args['max_q_len'],
                                                doc_stride=128,
                                                is_training=True)

features = features[:int(len(features) * args['ratio'])]
all_c_ids = torch.tensor([f.c_ids for f in features], dtype=torch.long)
data = TensorDataset(all_c_ids)
data_loader = DataLoader(data, shuffle=False, batch_size=args['batch_size'])



new_features = []

for batch in tqdm(data_loader, total=len(data_loader)):
    c_ids = batch[0]
    _, c_len = return_mask_lengths(c_ids)
    max_c_len = torch.max(c_len)
    c_ids = c_ids[:, :max_c_len].to(device)
    
    # sample latent variable K times
    for _ in range(args['k']):
        with torch.no_grad():
            _, _, zq, _, za = vae.prior_encoder(c_ids)
            batch_q_ids, batch_start, batch_end = vae.generate(zq, za, c_ids)

            all_input_ids, all_seg_ids, \
            all_input_mask, all_start, all_end = post_process(batch_q_ids, batch_start, batch_end, c_ids)

        for i in range(c_ids.size(0)):
            new_features.append(
                InputFeatures(
                    unique_id=None,
                    example_index=None,
                    doc_span_index=None,
                    tokens=None,
                    token_to_orig_map=None,
                    token_is_max_context=None,
                    input_ids=all_input_ids[i].cpu().tolist(),
                    input_mask=all_input_mask[i].cpu().tolist(),
                    c_ids=None,
                    context_tokens=None,
                    q_ids=None,
                    q_tokens=None,
                    answer_text=None,
                    tag_ids=None,
                    segment_ids=all_seg_ids[i].cpu().tolist(),
                    noq_start_position=None,
                    noq_end_position=None,
                    start_position=all_start[i].cpu().tolist(),
                    end_position=all_end[i].cpu().tolist(),
                    is_impossible=None))

dir_name = os.path.dirname(args['output_file'])
if not os.path.exists(dir_name):
    os.makedirs(dir_name)
with open(args['output_file'], "wb") as f:
    pickle.dump(new_features, f)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 48/48 [06:29<00:00,  8.12s/it]


In [ ]:
# tokenizer = BertTokenizer.from_pretrained(args['bert_model'])
for x in range(0,200):
    st = tokenizer.decode(new_features[x].input_ids)
    print(st)
    st1 = tokenizer.decode(new_features[x].input_ids[new_features[x].start_position:new_features[x].end_position])
    print(st1)
    print()

[CLS] what is the name of the sacred heart of the sacred heart? [SEP] architecturally, the school has a catholic character. atop the main building's gold dome is a golden statue of the virgin mary. immediately in front of the main building and facing it, is a copper statue of christ with arms upraised with the legend " venite ad me omnes ". next to the main building is the basilica of the sacred heart. immediately behind the basilica is the grotto, a marian place of prayer and reflection. it is a replica of the grotto at lourdes, france where the virgin mary reputedly appeared to saint bernadette soubirous in 1858. at the end of the main drive ( and in a direct line that connects through 3 statues and the gold dome ), is a simple, modern stone statue of mary. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [P

In [ ]:
f = open("/content/Info-HCVAE/data/squad/train-v1.1.json")
data = json.load(f)
print(data['data'][0]['paragraphs'][1]['context'])

নাগরিকদের দ্বারা প্রতিবছর নির্বাচিত হন এবং নিয়োগপ্রাপ্ত ম্যাজিস্ট্রেটদের সমন্বয়ে গঠিত সিনেটের পরামর্শ দিয়েছিলেন রোমান সরকারের নেতৃত্বে দুটি কনসাল। যেহেতু রোমান সমাজ আধুনিক মানদণ্ড দ্বারা অত্যন্ত শ্রেণিবদ্ধ ছিল, রোম সরকারের বিবর্তন তাত্ত্বিকদের মধ্যে লড়াই দ্বারা প্রচুর প্রভাবিত হয়েছিল, রোমের স্থল-অধিপতি অভিজাত, যিনি রোমের প্রতিষ্ঠার পূর্বসূরীর সন্ধান করেছেন এবং তাঁর পক্ষ থেকে আরও অনেক মানুষ ছিলেন নাগরিক-সাধারণের। সময়ের সাথে সাথে, প্যাট্রেসিদেরকে রোমের সর্বোচ্চ অফিসগুলিতে একচেটিয়া অধিকার প্রদান করা আইনগুলি বাতিল বা দুর্বল করা হয়েছিল এবং নেতৃস্থানীয় সুন্দরী পরিবারগুলি অভিজাত শ্রেণির পুরো সদস্য হয়ে ওঠে। প্রজাতন্ত্রের নেতারা একটি শক্তিশালী তিহ্য এবং নৈতিকতার বিকাশ করেছিলেন, যা জনসেবা ও শান্তি ও যুদ্ধের পৃষ্ঠপোষকতার প্রয়োজন, সামরিক ও রাজনৈতিক সাফল্যকে অনিচ্ছাকৃতভাবে সংযুক্ত করে। রোমের অনেক আইনী এবং আইনী কাঠামো পরে জাস্টিনিয়ান কোডে এবং পরে নেপোলিয়োনিক কোডে কোডেড এখনও আধুনিক দেশ রাষ্ট্র এবং আন্তর্জাতিক সংস্থাগুলিতে পুরো ইউরোপ এবং বিশ্বের বেশিরভাগ অংশ লক্ষ্য করা যায়।  
